In [ ]:
import psycopg2
from pymongo import MongoClient
from cassandra.cluster import Cluster
import redis
import mysql.connector
import yaml

with open('docker-compose.yml', 'r') as file:
    docker_config = yaml.safe_load(file)

# PostgreSQL connection
postgres_config = docker_config['services']['postgresql']
postgres_client = psycopg2.connect(
    host='localhost',  # or 'postgresql' if running inside Docker network
    database=postgres_config['environment']['POSTGRES_DB'],
    user=postgres_config['environment']['POSTGRES_USER'],
    password=postgres_config['environment']['POSTGRES_PASSWORD'],
    port=postgres_config['ports'][0].split(':')[0]
)

# MariaDB connection
mariadb_config = docker_config['services']['mariadb']
mariadb_client = mysql.connector.connect(
    host='localhost',  # or 'mariadb' if running inside Docker network
    database=mariadb_config['environment']['MYSQL_DATABASE'],
    user=mariadb_config['environment']['MYSQL_USER'],
    password=mariadb_config['environment']['MYSQL_PASSWORD'],
    port=mariadb_config['ports'][0].split(':')[0]
)

# Cassandra connection
cassandra_config = docker_config['services']['cassandra']
cassandra_client = Cluster(['localhost'],  # or ['cassandra'] if running inside Docker network
                        port=cassandra_config['ports'][0].split(':')[0])
cassandra_session = cassandra_client.connect()

# MongoDB connection
mongo_config = docker_config['services']['mongodb']
mongo_client = MongoClient(
    host='localhost',  # or 'mongodb' if running inside Docker network
    port=int(mongo_config['ports'][0].split(':')[0])
)

# Redis connection
redis_config = docker_config['services']['redis']
redis_client = redis.Redis(
    host='localhost',  # or 'redis' if running inside Docker network
    port=int(redis_config['ports'][0].split(':')[0])
)

In [13]:
# Simple sanity checks for each database connection

# PostgreSQL check
try:
    with postgres_client.cursor() as cursor:
        cursor.execute("SELECT 1")
        print("PostgreSQL connection successful")
except Exception as e:
    print(f"PostgreSQL connection failed: {e}")

# MariaDB check
try:
    with mariadb_client.cursor(buffered=True) as cursor:
        cursor.execute("SELECT 1")
        print("MariaDB connection successful")
except Exception as e:
    print(f"MariaDB connection failed: {e}")


# Cassandra check
try:
    cassandra_session.execute("SELECT release_version FROM system.local")
    print("Cassandra connection successful")
except Exception as e:
    print(f"Cassandra connection failed: {e}")

# MongoDB check
try:
    mongo_client.admin.command('ping')
    print("MongoDB connection successful")
except Exception as e:
    print(f"MongoDB connection failed: {e}")

# Redis check
try:
    redis_client.ping()
    print("Redis connection successful")
except Exception as e:
    print(f"Redis connection failed: {e}")


PostgreSQL connection successful
MariaDB connection successful
Cassandra connection successful
MongoDB connection successful
Redis connection successful
